# HW 3

Muyuan Zhang

u1430770

06/12/2023

In [ ]:
import pandas as pd
import numpy as np
import re
import requests
import time
from bs4 import BeautifulSoup
import statsmodels.formula.api as smf
import seaborn as sns

## Part 1

In [ ]:
headers = {"User-agent" : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15'}
ranks = []
title_lens = []
ages = []
points = []
comment_nums = []

def scrape(page_num):
    URL = 'https://news.ycombinator.com/news?p=' + str(page_num)
    page = requests.get(f'{URL}', headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    for rank in soup.find_all('span', class_='rank'):
        ranks.append(int(rank.get_text().strip().replace('.', '')))

    for title in soup.find_all('span', class_='titleline'):
        cur_title = re.sub("\(.*?\)|\[.*?\]","", title.text.strip()).strip()
        title_lens.append(len(cur_title))

    for row in soup.find_all('td', class_='subtext'):
        age = row.find("span", class_='age').find('a')

        if age is None:
            ages.append(float(0))
        elif 'minute' in age.text:
            ages.append(float(age.text.strip().replace(" minute ago", '').replace('minutes ago', ''))/60)
        elif 'day' in age.text:
            ages.append(float(age.text.strip().replace(" day ago", '').replace('days ago', ''))*24)
        else:
            ages.append(float(age.text.strip().replace(" hour ago", '').replace('hours ago', '')))

        score = row.find("span", class_='score')
        if score is None:
            points.append(0)
        else:
            points.append(int(score.text.strip().replace(' points', '').replace(' point', '')))

        comment = row.find_all('a')[-1]
        if 'comment' in comment.text:
            comment_nums.append(int(comment.text.strip().replace(u'\xa0', u' ').replace(' comments', '').replace('comment', '')))
        else:
            comment_nums.append(0)

    time.sleep(1)

for i in range(1, 6):
    scrape(i)

df = pd.DataFrame({'rank': ranks, 'length_of_title': title_lens, 'age_in_hours' : ages, 'points' : points, 'number_of_comments': comment_nums})
df

## Part 2

In [ ]:
results = smf.ols('rank ~ length_of_title', data=df).fit()
print(results.summary())

In [ ]:
results = smf.ols('rank ~ number_of_comments', data=df).fit()
print(results.summary())

In [ ]:
results = smf.ols('rank ~ age_in_hours', data=df).fit()
print(results.summary())

In [ ]:
results = smf.ols('rank ~ points', data=df).fit()
print(results.summary())

In [ ]:
results = smf.ols('rank ~ points + number_of_comments + length_of_title + age_in_hours', data=df).fit()
print(results.summary())

In [ ]:
results = smf.ols('rank ~ points + age_in_hours', data=df).fit()
print(results.summary())

In [ ]:
sns.regplot(x="points", y="rank", data=df).set(title="Rank vs Points")

In [ ]:
sns.regplot(x="age_in_hours", y="rank", data=df).set(title="Rank vs Age in Hours")

Q: Include at least 3 different regressions. Compare/contrast them. Which is the most useful. What are the R^2 scores, p-values for coefficients, and values of the coefficients - and what do these tell us?

A: R^2 score is a measure of the goodness of fit of a model. P-value decides to accept or reject a null hypothesis. Values of the coefficients are the values that multiply the predictor values.

Q: Are there linear relationships between any of the variables?

A: The `rank` ~ `age_in_hours` pair has the relatively strongest single linear relationship with a greatest R-squared values. The `rank` ~ `age_in_hours & points` model fits best among all the models.

## Part 3

In [ ]:
# Use logistic regression to attempt to classify whether or not 
# an article will be on the front page, given the other (non-rank) variables
df['is_on_front_page'] = [1 if x < 31 else 0 for x in df['rank']]

log_reg = smf.logit('is_on_front_page ~ age_in_hours', df).fit()
print(log_reg.summary())
sns.regplot(y='is_on_front_page', x='age_in_hours', logistic=True, data=df, ci=None)

In [ ]:
log_reg = smf.logit('is_on_front_page ~ points', df).fit()
print(log_reg.summary())
sns.regplot(y='is_on_front_page', x='points', logistic=True, data=df, ci=None)

In [ ]:
log_reg = smf.logit('is_on_front_page ~ points + age_in_hours', df).fit()
print(log_reg.summary())
p = log_reg.params
predictions = []

for index, row in df.iterrows() :
    predictions.append(p.Intercept + p.points * row.points + p.age_in_hours * row.age_in_hours)

df["my_predictions"] = predictions
sns.regplot(y='is_on_front_page', x='my_predictions', data=df, logistic=True)

ax = df.plot(x='rank', y='my_predictions', kind='scatter', c='is_on_front_page', colormap='viridis', figsize=[10,10])
print(df.head())

x=df['rank']
ax.plot(x, [p.Intercept + p.points + p.age_in_hours for x in range(1,151)])
ax.set_xlim([1, 151])

In [ ]:
predictions = []

for index, row in df.iterrows():
    predictions.append((row.points - 1) / (row.age_in_hours + 2)**1.8)

df["other_predictions"] = predictions
sns.regplot(y='is_on_front_page', x='other_predictions', data=df, logistic=True)

ax = df.plot(x='rank', y='other_predictions', kind='scatter', c='is_on_front_page', colormap='viridis', figsize=[10, 10])
print(df.head())

x = df['rank']
ax.plot(x, [1.5 for x in range(1,151)])
ax.set_xlim([1, 151])